# Loss 测试


In [23]:
import torch
from torch import nn
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
print(input)
print(target)

output = loss(input, target)
print(output)
output.backward()
print(output)

tensor([[ 0.2906,  0.7886, -0.8071,  0.6975, -0.4337],
        [ 0.8174,  0.0636, -1.0646,  0.6757,  0.4634],
        [-1.0926, -0.4392, -1.2230,  0.7750, -0.1133]], requires_grad=True)
tensor([1, 1, 3])
tensor(1.2372, grad_fn=<NllLossBackward0>)
tensor(1.2372, grad_fn=<NllLossBackward0>)


In [24]:
# Example of target with class probabilities
# If containing class probabilities, same shape as the input and each value should be between [0,1]
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
print(input)
print(target)
output = loss(input, target)
output.backward()
print(output)

tensor([[ 1.1117,  0.2177,  0.2326, -1.2596,  0.7871],
        [-0.9317,  0.7030,  0.2557,  0.8344, -0.0866],
        [-1.0184,  0.4584, -0.3879,  1.5941,  1.2739]], requires_grad=True)
tensor([[0.2037, 0.0211, 0.1943, 0.0515, 0.5295],
        [0.1375, 0.0930, 0.4348, 0.2061, 0.1286],
        [0.0290, 0.1263, 0.1008, 0.6414, 0.1025]])
tensor(1.4823, grad_fn=<DivBackward1>)


# 导包

In [1]:
import torch

from src.model import AntBeeClassifier
from src.dataset import ClassDirectoryDataset
from torch.optim import Adam
from torch.utils.data import DataLoader

# 优化器


In [7]:
num_epochs = 3
batch_size = 32
num_workers = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


创建dataset 和 dataloader

In [3]:
myClassDirectoryDataset = ClassDirectoryDataset("../data/hymenoptera_data/train", ["jpg"])
myDataloader = DataLoader(myClassDirectoryDataset,
                          batch_size=batch_size,
                          num_workers=num_workers,
                          drop_last=False,
                          shuffle=True,
                          pin_memory=True  # 如果使用GPU，启用此选项加速数据传输
)

img0, label0 = myClassDirectoryDataset[5]
print("img0.shape:", img0.shape)
print("label0_id:", label0)
print("label0:",myClassDirectoryDataset.classes[label0])

img0.shape: torch.Size([3, 224, 224])
label0_id: 0
label0: ants


创建模型、Optimizer

In [4]:
model = AntBeeClassifier(dropout_rate=0.2) # 1. build a model
model = model.to(device)

loss_fn = torch.nn.CrossEntropyLoss() # 2. define the loss
optimizer = Adam(model.parameters(), lr=0.001 , weight_decay=1e-4) # 3. do the optimize work
# 添加权重衰减防止过拟合

train


In [9]:

for epoch in range(num_epochs):
    train_loss = 0.0
    for inputs, labels in myDataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, labels) # 计算 交叉熵loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    print(train_loss)


3.4908050894737244
3.1987371146678925
2.9958401322364807
